In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-10-01'
final_date = '2022-10-31'
query_error_year = 2022
query_error_month = 10

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\10. OCTUBRE\DATA\ERROR OCTUBRE 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(110258, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10984\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
95104,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual
95869,95869,ac423215-3a95-4d3d-ac53-22b9d7190be4,Renovadora karla,B6GKT9YL4,2022-10-01,9.58,ERROR,Bank account information not available,2022-10-01,10,2022,NEW,SIN_CLABE,ac423215-3a95-4d3d-ac53-22b9d7190be4,2022-10-01,1,Actual
95868,95868,040c88e9-fff1-4518-a1a7-72fee20f9803,La bendicin,MGF9377FJ,2022-10-01,578.78,ERROR,Bank account information not available,2022-10-01,10,2022,NEW,SIN_CLABE,040c88e9-fff1-4518-a1a7-72fee20f9803,2022-10-01,1,Actual
96040,96040,cabe926f-2680-438f-be3f-20dcc664f09f,Blen castle,T3NSWVG5R,2022-10-01,0.01,ERROR,Bank account information not available,2022-10-01,10,2022,NEW,SIN_CLABE,cabe926f-2680-438f-be3f-20dcc664f09f,2022-10-01,1,Actual
96039,96039,c43dd1ff-4538-4351-841d-050b6372beeb,Abarrotes pimentel,YW2V2YHJ6,2022-10-01,32.83,ERROR,Bank account information not available,2022-10-01,10,2022,NEW,SIN_CLABE,c43dd1ff-4538-4351-841d-050b6372beeb,2022-10-01,1,Actual


In [9]:
query_error_current_month.shape

(14725, 17)

In [10]:
query_error_current_month['month'].unique()

array([10], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10984\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
79805,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN
78981,78981,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,VLHJ89MM7,2022-07-15,50.78,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN
75244,75244,2b78b265-6f3a-4677-b74c-433d3ad9b373,Abarrotes Victoria,SL7XKL2ND,2022-06-27,"1,735.37",ERROR,Error request by Backoffice,2022-09-02,9,2022,NEW,ERROR_USUAL,2b78b265-6f3a-4677-b74c-433d3ad9b373,2022-09-02,2,NaN
70770,70770,274c3150-39d9-47d3-a94d-ce6e188f759e,taqueria y jugueria Rosario,TP77RQRMV,2022-05-29,207.94,ERROR,Error request by Backoffice,2022-09-02,9,2022,NEW,ERROR_USUAL,274c3150-39d9-47d3-a94d-ce6e188f759e,2022-09-02,2,NaN
75750,75750,2b78b265-6f3a-4677-b74c-433d3ad9b373,Abarrotes Victoria,BCLDKQGMF,2022-06-30,339.69,ERROR,Error request by Backoffice,2022-09-02,9,2022,NEW,ERROR_USUAL,2b78b265-6f3a-4677-b74c-433d3ad9b373,2022-09-02,2,NaN


In [15]:
query_new_last_month.shape

(52, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\10. OCTOBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\10. OCTUBRE\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [23]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\09. SEPTEMBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\09. SEPTIEMBRE\QUERIES\*.csv', recursive=False)

In [24]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [25]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [26]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\08. AUGUST\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\08. AGOSTO\QUERIES\*.csv', recursive=False)

In [27]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [28]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [29]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
35907,DAILY,5ac94ce8-19ec-4b6a-82ef-2ef8fcb1826e,Estética Illiana,wendyiaa@hotmail.com,22249H9VW,2022-07-30,"1,054.06","Banco Mercantil del Norte, S.A.",2022-07-30 10:15:59,Wendy Illiana Arroyo Acosta,'072020001658164305,2022-07-30 08:08:18,BANORTE,"Wendy Illiana Arroyo Acosta,,1054.06,072020001...",BNTE 2022-07-30.csv,2022-07-30,TRAD
29351,DAILY,4a1c51be-6d4c-43c3-b0cf-d4943ebedec6,Alondra Sandoval,sandovalh.alondra@gmail.com,2224X37CX,2022-07-30,"1,212.17","Scotiabank Inverlat, S.A.",2022-07-30 10:42:11,Ma. del Socorro Hernandez Barboza,'044222020048286079,2022-07-30 08:07:32,BANORTE,"Ma. del Socorro Hernandez Barboza,,1212.17,044...",BNTE 2022-07-30.csv,2022-07-30,TRAD
66339,DAILY,a7affd7c-6fc4-43ec-8020-2be82192f77b,Carniceria La Mexicana,raul@villanueva.pro,222DTDXHR,2022-07-30,"4,342.75","BBVA Bancomer, S.A.",2022-07-30 10:44:55,Raul Villanueva Iglesias,'012180001965793453,2022-07-30 08:12:00,BANORTE,"Raul Villanueva Iglesias,,4342.75,012180001965...",BNTE 2022-07-30.csv,2022-07-30,TRAD
62924,DAILY,9e918960-d6c4-4329-bb35-d7405005b3f2,Julia quintero estudio de belleza,juliaquintero169@gmail.com,222FQTD75,2022-07-30,958.24,"Banco Nacional de México, S.A.",2022-07-30 10:22:06,Julia Quintero Ultimo,'002022700056571623,2022-07-30 08:11:34,BANORTE,"Julia Quintero Ultimo,,958.24,0020227000565716...",BNTE 2022-07-30.csv,2022-07-30,TRAD
73138,DAILY,b7f83713-89b7-4def-849f-a26520c80aab,Kmaleon Uniformes y Bordados,nperez@uniformeskmaleon.com,222GRTD93,2022-07-30,766.59,BBVA BANCOMER,2022-07-30 10:45:23,Elva Nidia Perez Zermeño,'012818001662148437,2022-07-30 08:12:45,BANORTE,"Elva Nidia Perez Zermeño,,766.59,0128180016621...",BNTE 2022-07-30.csv,2022-07-30,TRAD


In [30]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [31]:
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
35907,DAILY,5ac94ce8-19ec-4b6a-82ef-2ef8fcb1826e,Estética Illiana,wendyiaa@hotmail.com,22249H9VW,2022-07-30,"1,054.06","Banco Mercantil del Norte, S.A.",2022-07-30 10:15:59,Wendy Illiana Arroyo Acosta,'072020001658164305,2022-07-30 08:08:18,BANORTE,"Wendy Illiana Arroyo Acosta,,1054.06,072020001...",BNTE 2022-07-30.csv,2022-07-30,TRAD,Pago del día
29351,DAILY,4a1c51be-6d4c-43c3-b0cf-d4943ebedec6,Alondra Sandoval,sandovalh.alondra@gmail.com,2224X37CX,2022-07-30,"1,212.17","Scotiabank Inverlat, S.A.",2022-07-30 10:42:11,Ma. del Socorro Hernandez Barboza,'044222020048286079,2022-07-30 08:07:32,BANORTE,"Ma. del Socorro Hernandez Barboza,,1212.17,044...",BNTE 2022-07-30.csv,2022-07-30,TRAD,Pago del día
66339,DAILY,a7affd7c-6fc4-43ec-8020-2be82192f77b,Carniceria La Mexicana,raul@villanueva.pro,222DTDXHR,2022-07-30,"4,342.75","BBVA Bancomer, S.A.",2022-07-30 10:44:55,Raul Villanueva Iglesias,'012180001965793453,2022-07-30 08:12:00,BANORTE,"Raul Villanueva Iglesias,,4342.75,012180001965...",BNTE 2022-07-30.csv,2022-07-30,TRAD,Pago del día
62924,DAILY,9e918960-d6c4-4329-bb35-d7405005b3f2,Julia quintero estudio de belleza,juliaquintero169@gmail.com,222FQTD75,2022-07-30,958.24,"Banco Nacional de México, S.A.",2022-07-30 10:22:06,Julia Quintero Ultimo,'002022700056571623,2022-07-30 08:11:34,BANORTE,"Julia Quintero Ultimo,,958.24,0020227000565716...",BNTE 2022-07-30.csv,2022-07-30,TRAD,Pago del día
73138,DAILY,b7f83713-89b7-4def-849f-a26520c80aab,Kmaleon Uniformes y Bordados,nperez@uniformeskmaleon.com,222GRTD93,2022-07-30,766.59,BBVA BANCOMER,2022-07-30 10:45:23,Elva Nidia Perez Zermeño,'012818001662148437,2022-07-30 08:12:45,BANORTE,"Elva Nidia Perez Zermeño,,766.59,0128180016621...",BNTE 2022-07-30.csv,2022-07-30,TRAD,Pago del día


In [32]:
allfiles.shape

(8689809, 18)

# CRUCE 1 - MES ACTUAL

In [33]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,5ac94ce8-19ec-4b6a-82ef-2ef8fcb1826e,Estética Illiana,wendyiaa@hotmail.com,22249H9VW,2022-07-30,"1,054.06","Banco Mercantil del Norte, S.A.",2022-07-30 10:15:59,Wendy Illiana Arroyo Acosta,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,4a1c51be-6d4c-43c3-b0cf-d4943ebedec6,Alondra Sandoval,sandovalh.alondra@gmail.com,2224X37CX,2022-07-30,"1,212.17","Scotiabank Inverlat, S.A.",2022-07-30 10:42:11,Ma. del Socorro Hernandez Barboza,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,a7affd7c-6fc4-43ec-8020-2be82192f77b,Carniceria La Mexicana,raul@villanueva.pro,222DTDXHR,2022-07-30,"4,342.75","BBVA Bancomer, S.A.",2022-07-30 10:44:55,Raul Villanueva Iglesias,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,9e918960-d6c4-4329-bb35-d7405005b3f2,Julia quintero estudio de belleza,juliaquintero169@gmail.com,222FQTD75,2022-07-30,958.24,"Banco Nacional de México, S.A.",2022-07-30 10:22:06,Julia Quintero Ultimo,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,b7f83713-89b7-4def-849f-a26520c80aab,Kmaleon Uniformes y Bordados,nperez@uniformeskmaleon.com,222GRTD93,2022-07-30,766.59,BBVA BANCOMER,2022-07-30 10:45:23,Elva Nidia Perez Zermeño,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [34]:
cruce_1.shape

(8689809, 26)

In [35]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
23,DAILY,c4aeb0d4-1595-4b68-9b67-d2087d40bf89,Mini Super Keyla,lsaldana68.ls@gmail.com,226SYVVST,2022-07-30,220.78,"BanCoppel, S.A.",2022-07-30 13:47:57,Leonardo,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
24,DAILY,c4aeb0d4-1595-4b68-9b67-d2087d40bf89,Mini Super Keyla,lsaldana68.ls@gmail.com,226SYVVST,2022-07-30,220.78,"BanCoppel, S.A.",2022-07-30 13:47:57,Leonardo,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
47,DAILY,891e4cf0-72ee-4c21-9d30-1ef53a97ad5c,POP CLEAN,eric.narvaez.fuentes@gmail.com,22DB6Y2TQ,2022-07-30,81.93,"Banco Santander, S.A.",2022-07-30 13:48:53,Eric Adrian Narvaez Fuentes,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
48,DAILY,891e4cf0-72ee-4c21-9d30-1ef53a97ad5c,POP CLEAN,eric.narvaez.fuentes@gmail.com,22DB6Y2TQ,2022-07-30,81.93,"Banco Santander, S.A.",2022-07-30 13:48:53,Eric Adrian Narvaez Fuentes,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
49,DAILY,d89db37d-b896-43e3-891b-0c6984504370,Harbanos Tacos Arabes Francisco De,harbanosfm@outlook.com,22DFGY97X,2022-07-30,819.30,BBVA BANCOMER,2022-07-30 13:48:19,GERARDO MANRIQUE CUEVAS,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [36]:
duplicates_cruce_1_previo.shape

(208581, 26)

In [37]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10984\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
5582249,DAILY,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,huber_lafiel51@hotmail.com,2VD9LPLR6,2022-09-28,"2,874.72","Banco Regional de Monterrey, S.A.",2022-09-29 17:48:33,Huber Alberto Sandoval Vega,...,TRAD,Reprogramación,"2,874.72",2022-10-01,2022-10-01,10.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW
5585838,DAILY,99cf8a08-13d1-48c0-a900-83a119f84ef9,Eden,chemiguelangel62@gmail.com,49NW3R53L,2022-09-28,335.38,"BBVA Bancomer, S.A.",2022-09-29 15:17:14,Miguel Angel Che May,...,TRAD,Reprogramación,335.38,2022-10-01,2022-10-01,10.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,ERROR_USUAL,NEW
5587123,DAILY,88ce7837-c7ff-4747-bc7e-93f728049558,MiniSuper AVE FENIX,superavefenix25abr14@gmail.com,4RMQP82QK,2022-09-29,160.03,"Banco del Bajío, S.A.",2022-09-29 15:17:14,GABRIELA VALENZUELA CONTRERAS,...,TRAD,Pago del día,160.03,2022-10-01,2022-10-01,10.00,ERROR,DEV.SPEICUENTA CANCELADA,ERROR_USUAL,NEW
5594264,DAILY,84c2d8af-32e7-41dd-ba59-2eecf174538e,Canident,danaalin50@gmail.com,7L4SSJMWM,2022-09-28,250.10,"Banco Regional de Monterrey, S.A.",2022-09-29 17:56:22,Daniel Alejandro Canizales Avila,...,TRAD,Reprogramación,250.10,2022-10-01,2022-10-01,10.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW
5595770,DAILY,62b9782b-3bca-423d-9208-24b03971ddfd,Ediths Tours,jessica.edith.lopez@gmail.com,86PWKLNC6,2022-09-29,"1,264.88",BBVA BANCOMER,2022-09-29 17:56:22,Jessica Edith Cruz Lopez,...,TRAD,Pago del día,"1,264.88",2022-10-01,2022-10-01,10.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,ERROR_USUAL,NEW


In [38]:
cruce_1_final.shape

(9313, 26)

In [39]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [40]:
duplicates_cruce_1_final.shape

(0, 26)

# CRUCE 2 MES ACTUAL

In [41]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Pago del día
1,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Pago del día
2,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Reprogramación
3,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Reprogramación
4,95869,ac423215-3a95-4d3d-ac53-22b9d7190be4,Renovadora karla,B6GKT9YL4,2022-10-01,9.58,ERROR,Bank account information not available,2022-10-01,10,2022,NEW,SIN_CLABE,ac423215-3a95-4d3d-ac53-22b9d7190be4,2022-10-01,1,Actual,Not found,NaN


In [42]:
cruce_2.shape

(15572, 19)

In [43]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Pago del día
1,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Pago del día
2,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Reprogramación
3,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Reprogramación
9,95838,16c4e754-45f3-4562-ae3d-a7e745037fb6,Mini Super Fatima,ZQVWV8G4F,2022-09-30,96.30,ERROR,/21/CB Return Exceeds the limit of credits all...,2022-10-01,10,2022,NEW,ERROR_USUAL,16c4e754-45f3-4562-ae3d-a7e745037fb6,2022-10-01,1,Actual,FP,Pago del día


In [44]:
cruce_2_final.shape

(10160, 19)

In [45]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Pago del día
1,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Pago del día
2,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Reprogramación
3,95104,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,oraltech lab,2VD9LPLR6,2022-09-28,"2,874.72",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-10-01,10,2022,NEW,ERROR_USUAL,072fd99a-a2c7-47eb-8f48-6c6e08cf53de,2022-10-01,1,Actual,TRAD,Reprogramación
13,95834,cdb7a02f-d670-4cdd-b9d5-f33d189df646,El pescado Rey,KHTDVT5ZT,2022-09-30,160.98,ERROR,DEV.SPEICUENTA BLOQUEADA,2022-10-01,10,2022,NEW,ERROR_USUAL,cdb7a02f-d670-4cdd-b9d5-f33d189df646,2022-10-01,1,Actual,TRAD,Pago del día


In [46]:
duplicates_cruce_2_final.shape

(1435, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [47]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-10-01           0.00           0.03   
1                         2022-10-02       2,189.99           0.00   
2                         2022-10-03      29,864.79       5,007.40   
3                         2022-10-04           0.00      61,334.11   
4                         2022-10-05           0.00           0.00   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                   7,414.93           0.00    29,802.96       5,394.40   
1                       0.00       6,091.55   106,659.01       3,449.66   
2                   5,278.30       5,158.73   110,443.96      53,726.96   
3                   1,643.30           0.00    74,925.98       3,043.39   
4                   1,464.47           0.00   113,418.21       1,213.12   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0                42,612.32  
1               118,390.21  
2               209,480.14  
3               140,946.78  
4               116,095.80

In [48]:
group_tipo_pago.shape

(32, 8)

In [49]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-10-01           0.03    37,217.89       5,394.40   
1                       2022-10-02       2,189.99   106,659.01       9,541.21   
2                       2022-10-03      34,872.19   115,722.26      58,885.69   
3                       2022-10-04      61,334.11    76,569.28       3,043.39   
4                       2022-10-05           0.00   114,882.68       1,213.12   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0              42,612.32  
1             118,390.21  
2             209,480.14  
3             140,946.78  
4             116,095.80

In [50]:
group_esquema.shape

(32, 5)

# CRUCE 3 - MES ANTERIOR

In [51]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,5ac94ce8-19ec-4b6a-82ef-2ef8fcb1826e,Estética Illiana,wendyiaa@hotmail.com,22249H9VW,2022-07-30,"1,054.06","Banco Mercantil del Norte, S.A.",2022-07-30 10:15:59,Wendy Illiana Arroyo Acosta,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,4a1c51be-6d4c-43c3-b0cf-d4943ebedec6,Alondra Sandoval,sandovalh.alondra@gmail.com,2224X37CX,2022-07-30,"1,212.17","Scotiabank Inverlat, S.A.",2022-07-30 10:42:11,Ma. del Socorro Hernandez Barboza,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,a7affd7c-6fc4-43ec-8020-2be82192f77b,Carniceria La Mexicana,raul@villanueva.pro,222DTDXHR,2022-07-30,"4,342.75","BBVA Bancomer, S.A.",2022-07-30 10:44:55,Raul Villanueva Iglesias,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,9e918960-d6c4-4329-bb35-d7405005b3f2,Julia quintero estudio de belleza,juliaquintero169@gmail.com,222FQTD75,2022-07-30,958.24,"Banco Nacional de México, S.A.",2022-07-30 10:22:06,Julia Quintero Ultimo,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,b7f83713-89b7-4def-849f-a26520c80aab,Kmaleon Uniformes y Bordados,nperez@uniformeskmaleon.com,222GRTD93,2022-07-30,766.59,BBVA BANCOMER,2022-07-30 10:45:23,Elva Nidia Perez Zermeño,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [52]:
cruce_3.shape

(8689809, 26)

In [53]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10984\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2975149,DAILY,274c3150-39d9-47d3-a94d-ce6e188f759e,taqueria y jugueria Rosario,ortizemiliano557@gmail.com,6ZF47WRHF,2022-05-31,251.06,"BBVA Bancomer, S.A.",2022-08-31 15:22:38,Angelica,...,TRAD,Reprogramación,251.06,2022-09-02,2022-09-02,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2985669,DAILY,2b78b265-6f3a-4677-b74c-433d3ad9b373,Abarrotes Victoria,zamarripaquezada1985@gmail.com,BCLDKQGMF,2022-06-30,339.69,"BBVA Bancomer, S.A.",2022-08-31 15:22:40,1572500678,...,TRAD,Reprogramación,339.69,2022-09-02,2022-09-02,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
3030390,DAILY,2b78b265-6f3a-4677-b74c-433d3ad9b373,Abarrotes Victoria,zamarripaquezada1985@gmail.com,SL7XKL2ND,2022-06-27,"1,735.37","BBVA Bancomer, S.A.",2022-08-31 15:22:39,1572500678,...,TRAD,Reprogramación,"1,735.37",2022-09-02,2022-09-02,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
3033793,DAILY,274c3150-39d9-47d3-a94d-ce6e188f759e,taqueria y jugueria Rosario,ortizemiliano557@gmail.com,TP77RQRMV,2022-05-29,207.94,"BBVA Bancomer, S.A.",2022-08-31 15:22:38,Angelica,...,TRAD,Reprogramación,207.94,2022-09-02,2022-09-02,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
3070414,DAILY,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,compubyteizucar15@gmail.com,876H7BCPW,2022-07-18,191.65,"Banco Compartamos, S.A.",2022-09-01 10:33:20,Pedro Hernandez Evangelista,...,TRAD,Reprogramación,191.65,2022-09-01,2022-09-01,9.00,ERROR,DI,ERROR_USUAL,NEW


In [54]:
cruce_3_final.shape

(6, 26)

In [55]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [56]:
duplicates_cruce_3_final.shape

(0, 26)

# CRUCE 4 - MES ANTERIOR

In [57]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
1,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
2,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
3,78981,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,VLHJ89MM7,2022-07-15,50.78,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
4,78981,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,VLHJ89MM7,2022-07-15,50.78,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación


In [58]:
cruce_4.shape

(56, 19)

In [59]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
1,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
2,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
3,78981,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,VLHJ89MM7,2022-07-15,50.78,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
4,78981,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,VLHJ89MM7,2022-07-15,50.78,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación


In [60]:
cruce_4_final.shape

(10, 19)

In [61]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
1,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
2,79805,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,876H7BCPW,2022-07-18,191.65,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
3,78981,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,VLHJ89MM7,2022-07-15,50.78,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación
4,78981,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,VLHJ89MM7,2022-07-15,50.78,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,NaN,TRAD,Reprogramación


In [62]:
duplicates_cruce_4_final.shape

(6, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [63]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x         
esquema_query                           ERROR_USUAL      All
status_cubeta                                   NEW         
tipo_dispersion                                TRAD         
tipo_pago                            Reprogramación         
0                         2022-09-01         242.43   242.43
1                         2022-09-02       2,534.06 2,534.06
2                                All       2,776.49 2,776.49

In [64]:
group_tipo_pago_last_month.shape

(3, 3)

In [65]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount         
esquema_query                         ERROR_USUAL      All
status_cubeta                                 NEW         
tipo_pago                          Reprogramación         
0                       2022-09-01         727.29   727.29
1                       2022-09-02       2,534.06 2,534.06
2                              All       3,261.35 3,261.35

In [66]:
group_esquema_last_month.shape

(3, 3)

In [67]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL OCTUBRE 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()